Clone GitHub project

In [2]:
import os

# GitHub cloning...
user = "mss423"
repo = "ACS-LessIsMore"

# remove local directory if it already exists
if os.path.isdir(repo):
    !rm -rf {repo}

# clone repo
!git clone https://github.com/{user}/{repo}.git

Cloning into 'ACS-LessIsMore'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 249 (delta 136), reused 175 (delta 74), pack-reused 0 (from 0)
Receiving objects: 100% (249/249), 15.84 MiB | 8.01 MiB/s, done.
Resolving deltas: 100% (136/136), done.


Add codes to pwd

In [3]:
import sys

for dirpath, dirnames, filenames in os.walk("ACS-LessIsMore"):
        sys.path.append(dirpath)

Authenticate notebook for Google packages

In [4]:
# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

LOCATION = "us-central1"
PROJECT_ID = "synthetic-data-432701"
! gcloud services enable compute.googleapis.com aiplatform.googleapis.com storage.googleapis.com bigquery.googleapis.com --project {PROJECT_ID}

Operation "operations/acat.p2-29476227164-61dce8c8-a1d7-4d54-8957-35358052394c" finished successfully.


# Run experiments
## Load Data

In [ ]:
from load_data import *
import pandas as pd

'''
    Load the different dataset's sentences for downsampling
    FewRel --> synthetic n = 12800 (cut to 5k)
    SST2 --> synthetic n =  6000 (cut to 5k)
    ASTE --> synthetic n = 2000
    CrossNER --> synthetic n = 3000
'''

datadir = "/content/ACS-LessIsMore/datasets"
# savedir = "/content/ACS-LessIsMore/results"
dataset = 'sst2' # 'fewrel', 'sst2', 'aste', 'crossner'
savedir = os.path.join(datadir, "FewRel" if dataset == "fewrel" else dataset.upper() , "downsample")

# Load data_df, train and test
train_df, num_labels = load_train_data(datadir, dataset, synthetic=True)
test_df  = load_test_data(datadir, dataset)

#
num_labels = test_df['label'].nunique()
print(f"Number of training samples: {len(train_df)}")

In [14]:
# Downsample train_df to 1000 randomly selected samples for testing
if dataset != "aste":
    train_df = train_df.sample(n=5000, random_state=42)

## Run subsampling

In [ ]:
from acs import acs_sample
import numpy as np

# Ks = [len(train_df)//10]
Ks = np.round(np.linspace(100, len(train_df), 15)).astype(int)
# remove final entry as we dont need to run ACS to reobtain the dataset
Ks = Ks[:-1]

### ACS

In [ ]:
train_subsample_idx = acs_sample(train_df, Ks)

100%|██████████| 313/313 [01:04<00:00,  4.86it/s]


Processing for K = 100
Processing for K = 450
Processing for K = 800
Processing for K = 1150
Processing for K = 1500
Processing for K = 1850
Processing for K = 2200
Processing for K = 2550
Processing for K = 2900
Processing for K = 3250
Processing for K = 3600
Processing for K = 3950
Processing for K = 4300
Processing for K = 4650


Saving...

In [19]:
import pickle

save_name = "acs_subsample_idx.pkl"  # Choose your desired save name
save_path = os.path.join(savedir, save_name)  # Construct the full save path

# Check if a file with the same name already exists
if os.path.exists(save_path):
    print(f"Warning: File '{save_name}' already exists.")
else:
    # Save train_subsample_idx using pickle
    with open(save_path, 'wb') as f:
        pickle.dump(train_subsample_idx, f)
    print(f"Saved train_subsample_idx to '{save_path}'.")

Saved train_subsample_idx to '/content/ACS-LessIsMore/datasets/SST2/downsample/acs_subsample_idx.pkl'.
